In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to your dataset directory
dataset_dir = '/content/drive/MyDrive/Precog/Colab Notebooks/dataset'  # Replace with your dataset directory

# Define data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% for validation
    horizontal_flip=True,
    zoom_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    directory=dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_datagen = ImageDataGenerator(rescale=1./255)  # No augmentation for validation
validation_generator = validation_datagen.flow_from_directory(
    directory=dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Model Definition
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,  # Adjust epochs as needed
    verbose=1
)

# Evaluate the model
validation_generator.reset()  # Reset the validation generator
Y_pred = model.predict(validation_generator)
y_pred = np.round(Y_pred).astype(int)

print('Classification Report')
print(classification_report(validation_generator.classes, y_pred))

print('Confusion Matrix')
conf_mat = confusion_matrix(validation_generator.classes, y_pred)
sns.heatmap(conf_mat, annot=True, fmt='d')
plt.show()

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

Mounted at /content/drive
Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/10
50/50 [==============================] - 14s 257ms/step - loss: 0.6465 - accuracy: 0.6344 - val_loss: 0.5642 - val_accuracy: 0.7125
Epoch 2/10
50/50 [==============================] - 11s 218ms/step - loss: 0.5266 - accuracy: 0.7425 - val_loss: 0.5091 - val_accuracy: 0.7625
Epoch 3/10
50/50 [==============================] - 11s 221ms/step - loss: 0.4839 - accuracy: 0.7750 - val_loss: 0.4835 - val_accuracy: 0.7750
Epoch 4/10
50/50 [==============================] - 11s 222ms/step - loss: 0.4589 - accuracy: 0.7875 - val_loss: 0.4663 - val_accuracy: 0.7875
Epoch 5/10
50/50 [==============================] - 11s 223ms/step - loss: 0.4416 - accuracy: 0.7938 - val_loss: 0.4560 - val_accuracy: 0.7938
Epoch 6/10
50/50 [==============================] - 11s 221ms/step - loss: 0.4292 - accuracy: 0.8000 - val_loss: 0.4495 - val_accuracy: 0.7969
Epoch 7/10
50/50 [==============================] - 11s 220ms/step - loss: 0.4184 - accuracy: 0.8062 - val_loss: 0.4453 - val_accuracy: 0.7969
Epoch 8/10
50/50 [==============================] - 11s 220ms/step - loss: 0.4099 - accuracy: 0.8125 - val_loss: 0.4421 - val_accuracy: 0.8000
Epoch 9/10
50/50 [==============================] - 11s 222ms/step - loss: 0.4032 - accuracy: 0.8156 - val_loss: 0.4396 - val_accuracy: 0.8000
Epoch 10/10
50/50 [==============================] - 11s 222ms/step - loss: 0.3972 - accuracy: 0.8188 - val_loss: 0.4375 - val_accuracy: 0.8000
Classification Report
              precision    recall  f1-score   support

           0       0.81      0.80      0.81       200
           1       0.80      0.81      0.80       200

    accuracy                           0.80       400
   macro avg       0.80      0.80      0.80       400
weighted avg       0.80      0.80      0.80       400

Confusion Matrix

... (Heatmap of Confusion Matrix) ...

... (Accuracy and Loss Graphs) ...